# Phi-4 Mini ONNX lygiagrečių funkcijų kvietimų vadovas

This notebook demonstrates how to use Phi-4 Mini with ONNX Runtime GenAI for parallel function calling. Function calling allows the model to intelligently invoke external tools and APIs based on user requests.

## Overview

Šiame vadove sužinosite, kaip:
- Nustatyti Phi-4 Mini su ONNX Runtime GenAI
- Apibrėžti funkcijų schemas skrydžių ir viešbučių rezervavimui
- Naudoti vadovaujamą generavimą su Lark gramatika struktūrizuotam rezultatui
- Vykdyti lygiagrečius funkcijų kvietimus sudėtingoms kelionės užsakymo situacijoms

## Prerequisites

Prieš paleisdami šį užrašų knygelę, įsitikinkite, kad turite:
- Atsisiuntę Phi-4 Mini ONNX modelį
- Įdiegę `onnxruntime-genai` paketą
- Pagrindinį supratimą apie funkcijų kvietimo koncepcijas


## Žingsnis 1: Reikalingų bibliotekų importavimas

Pirmiausia importuosime reikalingas bibliotekas mūsų funkcijų kvietimo įgyvendinimui.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## 2 žingsnis: Modelio nustatymas ir konfigūracija

Dabar sukonfigūruosime Phi-4 Mini ONNX modelį. Įsitikinkite, kad kelią pakeitėte į savo tikrąjį modelio katalogą.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## 3 žingsnis: Konfigūruokite generavimo parametrus

Konfigūruokite generavimo parametrus, kad kontroliuotumėte modelio išvesties elgseną. Šie nustatymai užtikrina deterministinius ir koncentruotus atsakymus funkcijų kvietimams.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## 4 žingsnis: Apibrėžkite prieinamas funkcijas

Čia apibrėžiame funkcijas, kurias mūsų dirbtinio intelekto asistentas gali iškviesti. Šiame pavyzdyje turime dvi su kelionėmis susijusias funkcijas:
1. **booking_flight_tickets**: Skirta užsakyti skrydžius tarp oro uostų
2. **booking_hotels**: Skirta užsakyti viešbučių apgyvendinimą

Funkcijų apibrėžimai atitinka OpenAI funkcijų kvietimo schemos formatą.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Žingsnis 5: Gramatikos generavimo pagalbinės funkcijos

Šios pagalbinės funkcijos konvertuoja mūsų funkcijų apibrėžimus į Lark gramatikos formatą, kurį ONNX Runtime GenAI naudoja vadovaujamam generavimui. Tai užtikrina, kad modelis išvestų galiojančius funkcijų kvietimus teisingu JSON formatu.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## 6 žingsnis: Išbandykite gramatikos generavimą

Išbandykime mūsų gramatikos generavimo funkcijas, kad pamatytume, kaip jos konvertuoja mūsų įrankių apibrėžimus į tinkamą formatą.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## 7 žingsnis: Paruoškite sistemos promptą ir generatorių

Dabar sukursime sistemos promptą, kuris informuos modelį apie prieinamus įrankius ir nustatysime generatorių su vadovaujamais generavimo parametrais.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## 8 žingsnis: Inicializuokite Generator su giduojamu generavimu

Dabar sukursime generatorių su mūsų sukonfigūruotais parametrais ir pritaikysime Lark gramatiką giduojamam generavimui.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## 9 žingsnis: Išbandykite lygiagretinį funkcijų kvietimą

Dabar išbandykime mūsų nustatymą sudėtingu kelionės rezervavimo scenarijumi, kuriame reikia iškviesti kelias funkcijas.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Peržiūrėti suformatuotą naudotojo užklausą


### Sugeneruoti funkcijų kvietimus

Modelis dabar sugeneruos struktūruotus funkcijų kvietimus pagal mūsų vartotojo užklausą. Dėl nukreiptos generacijos, išvestis bus galiojantis JSON formatas, kurį galima tiesiogiai vykdyti.

**Tikėtina išvestis**: The model should generate function calls for:
1. `booking_flight_tickets` su informacija apie skrydį iš Pekino (PEK) į Paryžių (CDG)
2. `booking_hotels` su informacija apie apgyvendinimą Paryžiuje

Paleiskite žemiau esančią ląstelę, kad pamatytumėte tiesioginę generaciją:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Išvados

🎉 **Sveikiname!** Jūs sėkmingai įgyvendinote paralelinį funkcijų kvietimą su Phi-4 Mini naudodami ONNX Runtime GenAI.

### Ką išmokote:

1. **Modelio nustatymas**: Kaip sukonfigūruoti Phi-4 Mini su ONNX Runtime GenAI
2. **Funkcijos apibrėžimas**: Kaip apibrėžti funkcijų schemas AI funkcijų kvietimams
3. **Vadovaujama generacija**: Kaip naudoti Lark gramatiką struktūruotos išvesties generavimui
4. **Lygiagretūs funkcijų kvietimai**: Kaip tvarkyti sudėtingus užklausimus, reikalaujančius kelių funkcijų kvietimų

### Pagrindiniai privalumai:

- ✅ **Struktūruota išvestis**: Vadovaujama generacija užtikrina galiojančius JSON funkcijų kvietimus
- ✅ **Lygiagretinis apdorojimas**: Tvarkykite kelis funkcijų kvietimus per vieną užklausą
- ✅ **Aukštas našumas**: ONNX Runtime teikia optimizuotą inferenciją
- ✅ **Lanksčios schemos**: Lengva pridėti arba keisti funkcijų apibrėžimus

### Ištekliai:

- [Phi-4 Mini dokumentacija](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI dokumentacija](https://onnxruntime.ai/docs/genai/)
- [Geriausios funkcijų kvietimo praktikos](https://platform.openai.com/docs/guides/function-calling)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės apribojimas**:
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatizuoti vertimai gali turėti klaidų arba netikslumų. Originalus dokumentas gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Jei pateikiama kritinė informacija, rekomenduojamas profesionalus žmogaus vertimas. Mes neatsakome už jokius nesusipratimus ar klaidingus aiškinimus, kilusius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
